In [ ]:
from spectral import *
import spectral.io.envi as envi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.cluster import KMeans

In [ ]:
IMAGE_TO_OPEN = "C:\\Users\\Alex\\Desktop\\Q171\\Q171_1_irradiance"

img_raw = envi.open(IMAGE_TO_OPEN + ".hdr", IMAGE_TO_OPEN + ".raw")

img_raw_matrix = np.array(img_raw.load())

In [ ]:
def get_images_in_wavelength_range(hsi_image, band_start, band_end, bands):
    images = []
    for id, band in enumerate(bands):

        if band >= band_start and band <= band_end:
            images.append(hsi_image[:,:,id])
    return images


def get_mean_of_image_array(image_array : list):
    mean_of_each_band = [np.mean(image) for image in image_array]

    return np.mean(mean_of_each_band)

def get_ndvi_from_calibrated_hsi(image, bands):

    vnir_images = get_images_in_wavelength_range(image,650,680,bands)
    ir_images = get_images_in_wavelength_range(image,710,740,bands)

    mean_vnir_intensity = get_mean_of_image_array(vnir_images)
    mean_ir_intensity = get_mean_of_image_array(ir_images)

    return (mean_ir_intensity - mean_vnir_intensity) / (mean_ir_intensity + mean_vnir_intensity)

bands = img_raw.bands.centers
ndvi = get_ndvi_from_calibrated_hsi(calibrated_image, bands)
print(ndvi)

In [ ]:
# implementation from https://en.m.wikipedia.org/wiki/Otsu's_method

def otsu_intraclass_variance(image, threshold):
    """
    Otsu's intra-class variance.
    If all pixels are above or below the threshold, this will throw a warning that can safely be ignored.
    """
    return np.nansum(
        [
            np.mean(cls) * np.var(image, where=cls)
            #   weight   ·  intra-class variance
            for cls in [image >= threshold, image < threshold]
        ]
    )
    # NaNs only arise if the class is empty, in which case the contribution should be zero, which `nansum` accomplishes.


# Random image for demonstration:
image = np.random.randint(2, 253, size=(50, 50))

otsu_threshold = min(
    range(np.min(image) + 1, np.max(image)),
    key=lambda th: otsu_intraclass_variance(image, th),
)

#  what threshold?
print(otsu_intraclass_variance(calibrated_image, 1))